# Land cover and Land cover change from madmex api

In [124]:
# Python standard library
from datetime import datetime
from pprint import pprint

# External libraries
from datacube import Datacube
from datacube.storage import masking
import xarray as xr
import numpy as np
import folium

# MAdmex imports
from madmex.lcc.bitemporal.distance import BiChange
from madmex.util.xarray import to_float, to_int
from madmex.segmentation.bis import Segmentation
from madmex.overlay.extractions import zonal_stats_xarray
from madmex.io.vector_db import VectorDb
from madmex.modeling.supervised.lgb import Model
from madmex.lcc.bitemporal.distance import BiChange
from madmex.util.spatial import feature_transform

## What's the point of doing that with the API if there's the CLI?

- Build understanding
- Debug individual components
- ... 


## Summary of the steps

- Load data
- Prepare features
- Run segmentation
- Classify segmentation geometries
- Detect spectral changes and assign labels to pre and post
- Visualize some results


## Load data for a small study area

Here we select a small study area near Guadalaja, Jalisco. We'll compare 2014 land cover with 2017, so we load the two years as separate geoarrays.

In [85]:
dc = Datacube(app = 'jalisco_lcc')
sp_query = {'product': 'ls8_espa_mexico',
            'x': (-104, -103.6),
            'y': (20, 20.4)}
geoarray_pre = dc.load(**sp_query,
                       time=('2014-01-01', '2014-12-31'),
                       group_by='solar_day')
geoarray_post = dc.load(**sp_query,
                       time=('2017-01-01', '2017-12-31'),
                       group_by='solar_day')
geoarray_pre

<xarray.Dataset>
Dimensions:   (time: 4, x: 1407, y: 1497)
Coordinates:
  * time      (time) datetime64[ns] 2014-01-20T17:19:17.728594 ...
  * y         (y) float64 9.398e+05 9.398e+05 9.397e+05 9.397e+05 9.397e+05 ...
  * x         (x) float64 2.292e+06 2.292e+06 2.292e+06 2.292e+06 2.292e+06 ...
Data variables:
    blue      (time, y, x) int16 567 605 518 492 513 396 424 434 417 420 414 ...
    green     (time, y, x) int16 793 851 742 706 713 574 572 566 554 554 543 ...
    red       (time, y, x) int16 935 1039 858 792 818 590 608 606 587 584 ...
    nir       (time, y, x) int16 2227 2353 2278 2350 2471 2255 2000 1861 ...
    swir1     (time, y, x) int16 2936 3007 2687 2766 2582 2184 2212 2239 ...
    swir2     (time, y, x) int16 2093 2119 1841 1818 1692 1432 1464 1523 ...
    pixel_qa  (time, y, x) int16 322 322 322 322 322 322 322 322 322 322 322 ...
Attributes:
    crs:      PROJCS["unnamed",GEOGCS["WGS 84",DATUM["unknown",SPHEROID["WGS8...

## Prepare features

These features will be used for the segmentation, the classification and the change detection. This step is the equivalent to running the `antares apply_recipe` command line, except that the geoarrays containing the features are kept in memory instead of being registered as new datacube products.

In [86]:
def ls8_pp(sr):
    clear = masking.make_mask(sr.pixel_qa, cloud=False, cloud_shadow=False,
                                  snow=False)
    sr_1 = sr.where(clear)
    sr_1 = sr_1.drop('pixel_qa')
    sr_1 = sr_1.apply(func=to_float, keep_attrs=True)
    # Compute vegetation indices
    sr_1['ndvi'] = ((sr_1.nir - sr_1.red) / (sr_1.nir + sr_1.red)) * 10000
    sr_1['ndvi'].attrs['nodata'] = -9999
    sr_1['ndmi'] = ((sr_1.nir - sr_1.swir1) / (sr_1.nir + sr_1.swir1)) * 10000
    sr_1['ndmi'].attrs['nodata'] = -9999
    # Run temporal reductions and rename DataArrays
    sr_mean = sr_1.mean('time', keep_attrs=True, skipna=True)
    sr_mean.rename({'blue': 'blue_mean',
                    'green': 'green_mean',
                    'red': 'red_mean',
                    'nir': 'nir_mean',
                    'swir1': 'swir1_mean',
                    'swir2': 'swir2_mean',
                    'ndmi': 'ndmi_mean',
                    'ndvi': 'ndvi_mean'}, inplace=True)
    # Compute min/max/std only for vegetation indices
    ndvi_max = sr_1.ndvi.max('time', keep_attrs=True, skipna=True)
    ndvi_max = ndvi_max.rename('ndvi_max')
    ndvi_max.attrs['nodata'] = -9999
    ndvi_min = sr_1.ndvi.min('time', keep_attrs=True, skipna=True)
    ndvi_min = ndvi_min.rename('ndvi_min')
    ndvi_min.attrs['nodata'] = -9999
    # ndmi
    ndmi_max = sr_1.ndmi.max('time', keep_attrs=True, skipna=True)
    ndmi_max = ndmi_max.rename('ndmi_max')
    ndmi_max.attrs['nodata'] = -9999
    ndmi_min = sr_1.ndmi.min('time', keep_attrs=True, skipna=True)
    ndmi_min = ndmi_min.rename('ndmi_min')
    ndmi_min.attrs['nodata'] = -9999
    # Load terrain metrics using same spatial parameters than sr
    dc = Datacube(app = 'feature_creation')
    terrain = dc.load(product='srtm_cgiar_mexico', like=sr,
                      time=(datetime(1970, 1, 1), datetime(2018, 1, 1)))
    dc.close()
    # Merge dataarrays
    combined = xr.merge([sr_mean.apply(to_int),
                         to_int(ndvi_max),
                         to_int(ndvi_min),
                         to_int(ndmi_max),
                         to_int(ndmi_min),
                         terrain])
    combined.attrs['crs'] = sr.attrs['crs']
    return combined

features_pre = ls8_pp(geoarray_pre)
features_post = ls8_pp(geoarray_post)

/home/madmex_user/.virtualenvs/antares/lib/python3.5/site-packages/datacube/api/query.py:92: FutureWarning: casting an xarray.Dataset to a boolean will change in xarray v0.11 to only include data variables, not coordinates. Cast the Dataset.variables property instead to preserve existing behavior in a forwards compatible manner.
  if like:
/home/madmex_user/.virtualenvs/antares/lib/python3.5/site-packages/datacube/api/query.py:92: FutureWarning: casting an xarray.Dataset to a boolean will change in xarray v0.11 to only include data variables, not coordinates. Cast the Dataset.variables property instead to preserve existing behavior in a forwards compatible manner.
  if like:


## Load training data and Train model

In [87]:
# TODO: This class doesn't make much sense (but hard to remove without breaking anything)
loader = VectorDb()
# TODO: If class is kept, methods should at least be static
fc_train = loader.load_training_from_dataset(dataset=features_pre,
                                             training_set='jalisco_bits', sample=0.3)
# Evaluate the generator (safe to do here since the data is relatively small)
fc_train = list(fc_train)
pprint(fc_train[0]) # The property here refer to a unique database id for the class, not very meaningful
# here but convenient for handling labels in the database

{'geometry': {'coordinates': [[[2309914.0997946323, 924871.4525267063],
                               [2309929.036112618, 924871.1556520322],
                               [2309928.9369879263, 924866.1473473478],
                               [2309938.894533374, 924865.949432106],
                               [2309938.7954092147, 924860.9411273025],
                               [2309948.752954847, 924860.743213052],
                               [2309948.6538312286, 924855.7349081382],
                               [2309953.632604171, 924855.6359514053],
                               [2309953.3352344055, 924840.6110353662],
                               [2309958.3140079817, 924840.5120790553],
                               [2309958.214885093, 924835.5037732747],
                               [2309968.1724324855, 924835.3058614375],
                               [2309968.0733101415, 924830.2975555492],
                               [2309973.0520839687, 924830.1986000116],

### Extract training data and train model

We'll just use default parameters for now

In [88]:
# Training data correspond to the year 2015, closer in time to 2014 than to 2017, we therefore train using
# the pre features
X_train, y_train = zonal_stats_xarray(features_pre, fc_train, 'class')

lgb_model = Model()
lgb_model.fit(X_train, y_train)

## Run segmentation

In [89]:
s_bands = ['ndvi_mean', 'ndmi_mean']
Segmenter_pre = Segmentation.from_geoarray(features_pre[s_bands], t=20)
Segmenter_post = Segmentation.from_geoarray(features_post[s_bands], t=20)
Segmenter_pre.segment()
Segmenter_post.segment()

  Anticipated allocation: 82276KB
    ...allocated
  thresh 1
  thresh 2
  thresh 3
  thresh 4
  thresh 5
  thresh 6
  thresh 7
  thresh 8
  thresh 9
  thresh 10
  thresh 11
  thresh 12
  thresh 13
  thresh 14
  thresh 15
  thresh 16
  thresh 17
  thresh 18
  thresh 19
  thresh 20
  Anticipated allocation: 82276KB
    ...allocated
  thresh 1
  thresh 2
  thresh 3
  thresh 4
  thresh 5
  thresh 6
  thresh 7
  thresh 8
  thresh 9
  thresh 10
  thresh 11
  thresh 12
  thresh 13
  thresh 14
  thresh 15
  thresh 16
  thresh 17
  thresh 18
  thresh 19
  thresh 20


Extract feature collections from the segmentation instances

In [90]:
Segmenter_pre.polygonize(crs_out=None)
Segmenter_post.polygonize(crs_out=None)
# len(list(Segmenter.fc))
# 142354
# Sounds reasonable considering the array is about 1.5 million pixels. Average object size is therefore 1 ha
fc_seg_pre = list(Segmenter_pre.fc)
fc_seg_post = list(Segmenter_post.fc)
pprint(fc_seg_pre[0])

{'geometry': {'coordinates': [[(2291520.0, 939800.0),
                               (2291520.0, 939680.0),
                               (2291550.0, 939680.0),
                               (2291550.0, 939650.0),
                               (2291580.0, 939650.0),
                               (2291580.0, 939680.0),
                               (2291610.0, 939680.0),
                               (2291610.0, 939800.0),
                               (2291520.0, 939800.0)]],
              'type': 'Polygon'},
 'properties': {'id': 46678.0},
 'type': 'Feature'}


## Extract features under geometries


In [91]:
X_pre, _ = zonal_stats_xarray(features_pre, fc_seg_pre, 'id')
X_post, _ = zonal_stats_xarray(features_post, fc_seg_post, 'id')

X_pre.shape

(243983, 15)

## Predict land cover

In [114]:
y_pre = lgb_model.predict(X_pre)
y_post = lgb_model.predict(X_post)

fc_pred_pre = [(x[0]['geometry'], x[1]) for x in zip(fc_seg_pre, y_pre)]
fc_pred_post = [(x[0]['geometry'], x[1]) for x in zip(fc_seg_pre, y_post)]

pprint(fc_pred_pre[0])

/home/madmex_user/.virtualenvs/antares/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/madmex_user/.virtualenvs/antares/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


({'coordinates': [[(2291520.0, 939800.0),
                   (2291520.0, 939680.0),
                   (2291550.0, 939680.0),
                   (2291550.0, 939650.0),
                   (2291580.0, 939650.0),
                   (2291580.0, 939680.0),
                   (2291610.0, 939680.0),
                   (2291610.0, 939800.0),
                   (2291520.0, 939800.0)]],
  'type': 'Polygon'},
 155)


## Run change detection

We now have two land cover maps, we can try detecting changes between the two dates and then label these changes. Similarly to the segmentation, we'll use mean NDVI and mean NDVI as a basis for detecting changes.

In [118]:
lcc_pre = BiChange.from_geoarray(features_pre[s_bands], threshold = 3000)
lcc_post = BiChange.from_geoarray(features_post[s_bands])
lcc_pre.run(lcc_post)
print(lcc_pre.change_array.shape)
print(lcc_pre.change_array.sum())

(1497, 1407)
18500


In [136]:
lcc_pre.filter_mmu(5000)
print(lcc_pre.change_array.sum())
fc_change = lcc_pre.label_change(fc_pred_pre, fc_pred_post)
fc_change = lcc_pre.filter_no_change(fc_change)
print(len(fc_change))

16272
1195


## Visualize change results

In [142]:
fc_change_valid = [{'type': 'feature', 'geometry': x[0]}
                   for x in fc_change]
fc_change_valid = [feature_transform(x, crs_out='+proj=longlat', crs_in=lcc_pre.crs)
                   for x in fc_change_valid]
fc_change_valid = {"type": "FeatureCollection",
                   "features": fc_change_valid}

m = folium.Map(location=[21, -102],
               tiles='OpenStreetMap',
               zoom_start=7)
m.choropleth(geo_data=fc_change_valid, line_color='blue',
              line_weight=3)

m